# 自动微分

假设我们想对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导

In [1]:
import mindspore
import mindspore.numpy as mnp
from mindspore import ms_function

x = mnp.arange(4.0)
x

Tensor(shape=[4], dtype=Float32, value= [ 0.00000000e+00,  1.00000000e+00,  2.00000000e+00,  3.00000000e+00])

在我们计算$y$关于$\mathbf{x}$的梯度之前，我们需要介绍一下MindSpore的自动微分实现方式

MindSpore现有版本同时支持静态图和动态图，即GRAPH_MODE和PYNATIVE_MODE，为了将二者统一，整体进行自动微分的方式为静态图方式，即：
将函数视为一个完整的计算图，先进行编译，后执行。因此不像Pytorch一样将梯度grad直接绑定在Tensor上，而是整图运算后，再通过取梯度的算子进行梯度的提取。（动态图模式也并非纯Python执行，二者均采取静态图的策略）因此，和Pytorch有如下差异：


1. 想要自动微分的函数需要显式注册为function
2. 需要通过`ops.GradOperation`算子来获取梯度

现在让我们计算$y$

In [2]:
def forward(x):
    return 2 * mnp.dot(x, x)

y = forward(x)
y

Tensor(shape=[], dtype=Float32, value= 28)

通过调用`ops.GradOperation`算子来自动计算`y`关于`x`每个分量的梯度

In [3]:
import mindspore.ops as ops

grad_all = ops.GradOperation(get_all=True)
x_grad = grad_all(forward)(x)[0]
x_grad

[WARNING] OPTIMIZER(2932918,7f564ef4b740,python):2021-10-29-07:43:43.853.641 [mindspore/ccsrc/frontend/optimizer/ad/dfunctor.cc:803] GetPrimalUser] J operation has no relevant primal call in the same graph. Func graph: 1_after_grad.2, J user: 1_after_grad.2:forward{[0]: 3, [1]: args0}


Tensor(shape=[4], dtype=Float32, value= [ 0.00000000e+00,  4.00000000e+00,  8.00000000e+00,  1.20000000e+01])

In [4]:
x_grad == 4 * x

Tensor(shape=[4], dtype=Bool, value= [ True,  True,  True,  True])

现在让我们计算`x`的另一个函数

In [5]:
def forward(x):
    return x.sum()

In [6]:
x_grad = grad_all(forward)(x)[0]
x_grad

[WARNING] OPTIMIZER(2932918,7f564ef4b740,python):2021-10-29-07:43:43.965.997 [mindspore/ccsrc/frontend/optimizer/ad/dfunctor.cc:803] GetPrimalUser] J operation has no relevant primal call in the same graph. Func graph: 44_after_grad.11, J user: 44_after_grad.11:forward{[0]: 12, [1]: args0}


Tensor(shape=[4], dtype=Float32, value= [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00])

深度学习中
，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和

In [7]:
def forward(x):
    y = x * x
    return y.sum()

In [8]:
x_grad = grad_all(forward)(x)[0]
x_grad

[WARNING] OPTIMIZER(2932918,7f564ef4b740,python):2021-10-29-07:43:44.005.001 [mindspore/ccsrc/frontend/optimizer/ad/dfunctor.cc:803] GetPrimalUser] J operation has no relevant primal call in the same graph. Func graph: 70_after_grad.17, J user: 70_after_grad.17:forward{[0]: 18, [1]: args0}


Tensor(shape=[4], dtype=Float32, value= [ 0.00000000e+00,  2.00000000e+00,  4.00000000e+00,  6.00000000e+00])

将某些计算移动到记录的计算图之外

In [9]:
def forward(x):
    y = x * x
    u = ops.stop_gradient(y)
    z = u * x
    return z, u

z, u = forward(x)
x_grad = grad_all(forward)(x)[0]
x_grad == u

[WARNING] OPTIMIZER(2932918,7f564ef4b740,python):2021-10-29-07:43:44.036.291 [mindspore/ccsrc/frontend/optimizer/ad/dfunctor.cc:803] GetPrimalUser] J operation has no relevant primal call in the same graph. Func graph: 112_after_grad.24, J user: 112_after_grad.24:forward{[0]: 25, [1]: args0}


Tensor(shape=[4], dtype=Bool, value= [ True,  True,  True,  True])

In [10]:
def forward(x):
    y = x * x
    return y.sum()
x_grad = grad_all(forward)(x)[0]
x_grad == 2 * x

[WARNING] OPTIMIZER(2932918,7f564ef4b740,python):2021-10-29-07:43:44.094.246 [mindspore/ccsrc/frontend/optimizer/ad/dfunctor.cc:803] GetPrimalUser] J operation has no relevant primal call in the same graph. Func graph: 136_after_grad.33, J user: 136_after_grad.33:forward{[0]: 34, [1]: args0}


Tensor(shape=[4], dtype=Bool, value= [ True,  True,  True,  True])

即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

In [11]:
import numpy as np

def f(a):
    b = a * 2
    while mnp.norm(b) < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = mindspore.Tensor(np.random.randn(), mindspore.float32)
d = f(a)
a_grad = grad_all(f)(a)[0]

a_grad == d / a

[WARNING] OPTIMIZER(2932918,7f564ef4b740,python):2021-10-29-07:43:44.455.326 [mindspore/ccsrc/frontend/optimizer/ad/dfunctor.cc:803] GetPrimalUser] J operation has no relevant primal call in the same graph. Func graph: 374_178_after_grad.56, J user: 374_178_after_grad.56:f{[0]: 57, [1]: args0}


Tensor(shape=[], dtype=Bool, value= True)